### capture required trained model




In [ ]:
!pip install keras-video-generators

import numpy as np
import tensorflow as tf
from keras.applications import VGG16
from keras import models
from keras_video import VideoFrameGenerator
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import model_from_json
import matplotlib.pyplot as plt
from tensorflow.keras.models import model_from_json
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

model_path = 'model.json' # the path of json file of trained model
weight_path = 'model_weights.h5' # the path of weight file of trained model

cls={'Boli_Khela':0, 'Kabaddi':1 ,'Kho Kho':2, 'Lathi Khela':3, 'Nouka Baich':4} # categorical values of classes of the dataset


def get_frames(filename):
  SIZE = (128, 128)
  CHANNELS = 3
  NBFRAME = 20
  data_aug = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
  test=VideoFrameGenerator(glob_pattern=filename,nb_frames=NBFRAME,shuffle=False, batch_size=1, 
                           target_shape=SIZE, transformation=data_aug, use_frame_cache=False)
  return test

def get_model(model_path,weight_path):  
  json_file = open(model_path, 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  loaded_model = model_from_json(loaded_model_json)
  # load weights into new model
  loaded_model.load_weights(weight_path)
  return loaded_model



### Predict function

In [ ]:

def predict_class(filename, model_path=model_path, weight_path=weight_path,classes=cls):
    video_frames = get_frames(filename)
    model=get_model(model_path,weight_path)
    Y_pred = model.predict_generator(video_frames, 1)
    y_pred = np.argmax(Y_pred, axis=1)
    label = [k for k, v in classes.items() if v == y_pred[0]][0]
    return label

## flask webpage

In [ ]:
!mkdir "static/uploads"

In [4]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))
# click the generated link while running the next cell

https://tq6jpkvb9f-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
import os
# from app import app
import urllib.request
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename
from flask import Flask

UPLOAD_FOLDER = 'static/uploads/'

app = Flask(__name__)
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024


ALLOWED_EXTENSIONS = set(['mp4', 'avi'])

def allowed_file(filename):
	return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
	
@app.route('/')
def upload_form():
	return render_template('new_template.html')

def get_class(filename):
  path='/content/static/uploads/'
  cls=predict_class(path+filename)
  return cls

@app.route('/', methods=['POST'])
def upload_video():
    if 'file' not in request.files:
        flash('No file part')
        return render_template('new_template.html')
    file = request.files['file']
    if file.filename == '':
        flash('No video selected for uploading')
        return render_template('new_template.html')
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        flash('Video is successfully uploaded and displayed below')
        cls=get_class(filename)
        return render_template('new_template.html', filename=filename, cls='Class Label: '+cls)

    else:
        
        flash('Allowed video types are -> mp4, avi')
       
        return render_template('new_template.html')

if __name__ == "__main__":
    app.run()